In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

# Dataset
dataset = [
    {"input": "මම යනවා.", "target": "මම යමි."},
    {"input": "අපි යනවා.", "target": "අපි යමු."},
    {"input": "මම ගෙදර යන්නේ ය.", "target": "මම ගෙදර යන්නෙමි."},
    {"input": "මම පන්සල් යන්න පිටත් වුනේ ය.", "target": "මම පන්සල් යන්න පිටත් වුනෙමි."},
    {"input": "මම ගවේෂණය කරන්නේ ය.", "target": "මම ගවේෂණය කලෙමි."},
    {"input": "අපි ගවේෂණය කරන්නෝ ය.", "target": "අපි ගවේෂණය කලෙමු."},
    {"input": "මම පරීකෂණයට ගිහින් නදි එක්ක පන්සල් යනවා.", "target": "මම පරීකෂණයට ගිහින් නදි එක්ක පන්සල් යමි."},
    {"input": "අපි රීට පස්සෙ ගවේශනය  කරනවා  මොකක්ද  අවුල කියලා.", "target": "අපි රීට පස්සෙ ගවේශනය  කරමු  මොකක්ද  අවුල කියලා."},
    {"input": "මම නාන්න යනවා.", "target": "මම නාන්න යමි."},
    {"input": "අපි කන්න කෑම කාමරයට යන්න ලෑස්ති වුනෝ ය.", "target": "අපි කන්න කෑම කාමරයට යන්න ලෑස්ති වුනෙමි."},
    {"input": "මම කන්න යනවා.", "target": "මම කන්න යමි."},
    {"input": "අපි පාඩම් කරනවා.", "target": "අපි පාඩම් කරමු."},
    {"input": "ඔහු ගමට යන්නෙමි.", "target": "ඔහු ගමට  යන්නේ ය."},
    {"input": "අපි ගමට යන්න පිටත් වුනෝ ය.", "target": "අපි ගමට යන්න පිටත් වුනෙමු."},
    {"input": "මම කෝපි හදනවා.", "target": "මම කෝපි හදමි."},
    {"input": "ඔබ ආපසු ආවෙමි.", "target": "ඔබ ආපසු ආවේ ය."},
    {"input": "අපි පාසලට යන්නේ ය.", "target": "අපි පාසලට යන්නෙමු."},
    {"input": "ඔහු කාමරයට යමි.", "target": "ඔහු කාමරයට යනවා."},
    {"input": "අපි කන්න යමි.", "target": "අපි කන්න යමු."},
    {"input": "මම ඉගෙන ගැනීමේ වැඩ කරන්නේ ය.", "target": "මම ඉගෙන ගැනීමේ වැඩ කරෙමි."},
    {"input": "ඔහු පොත් කියවමි.", "target": "ඔහු පොත් කියවනවා."},
    {"input": "අපි ගීත ගායන කරනවා.", "target": "අපි ගීත ගායන කරමු."},
    {"input": "ඔවුන් ගමන ආරම්භ කරනවා.", "target": "ඔවුන් ගමන ආරම්භ කරති."},
    {"input": "මම නිවසට යනවා.", "target": "මම නිවසට යමි."},
    {"input": "ඔවුන් මිතුරන් හමු වූවේ ය.", "target": "ඔවුන් මිතුරන් හමු වූවෝ ය."},
    {"input": "අපි නිවසට ගොස් විවේක ගන්නවා.", "target": "අපි නිවසට ගොස් විවේක ගනිමු."},
    {"input": "මම ජායාරූප ගන්න යනවා.", "target": "මම ජායාරූප ගන්න යමි."},
    {"input": "ඔහු උයනේ ගමන් කරමි.", "target": "ඔහු උයනේ ගමන් කරනවා."},
    {"input": "අපි පසු ගිය දවස ගැන කතා කරනවා.", "target": "අපි පසු ගිය දවස ගැන කතා කරමු."},
    {"input": "ඔහු විශ්වාසයෙන් කතා කරමි.", "target": "ඔහු විශ්වාසයෙන් කතා කරනවා."},
    {"input": "මම යාලුවාට උදව් කරනවා.", "target": "මම යාලුවාට උදව් කරමි."},
    {"input": "අපි ගමන සූදානම් කරනවා.", "target": "අපි ගමන සූදානම් කරන්නෙමු."},
    {"input": "ඔවුන් විහිලු කරනවා.", "target": "ඔවුන් විහිලු කරති."},
    {"input": "මම සංගීතය අසමින් විවේක ගන්නවා.", "target": "මම සංගීතය අසමින් විවේක ගනිමි."},
    {"input": "අපි කවියක් රචනා කරනවා.", "target": "අපි කවියක් රචනා කරමු."},
    {"input": "ඔහු ඉතිහාසය කියවන්න යමි.", "target": "ඔහු ඉතිහාසය කියවන්න යනවා."},
    {"input": "මම උයනට ගොස් මල් සහ ගෙඩි බලනවා.", "target": "මම උයනට ගොස් මල් සහ ගෙඩි බලමි."},
    {"input": "අපි  උත්සවයට සහභාගී වෙනවා.", "target": "අපි උත්සවයට සහභාගී වන්නෙමු."},
    {"input": "ඔහු නිවැරදි මතයක් දක්වති.", "target": "ඔහු නිවැරදි මතයක් දක්වයි."},
    {"input": "මම කාර්යාලයට පිටත් වෙනවා.", "target": "මම කාර්යාලයට පිටත් වන්නෙමි."},
    {"input": "අපි උත්සවයට සූදානම් වෙනවා.", "target": "අපි උත්සවයට සූදානම් වන්නෙමු."},
    {"input": "ඔහු ගමන ආරම්භ කරති.", "target": "ඔහු ගමන ආරම්භ කරයි."},
    {"input": "මම ගමන් කරනවා.", "target": "මම ගමන් කරමි."},
    {"input": "අපි ප්‍රදර්ශනය බලනවා.", "target": "අපි ප්‍රදර්ශනය බලමු."},
    {"input": "ඔහු මල්වත්තේ සිටිති.", "target": "ඔහු මල්වත්තේ සිටියි."},
    {"input": "අපි නිවැරදි මගින් යනවා.", "target": "අපි නිවැරදි මගින් යමු."},
    {"input": "මම ඉගෙනීමේ පෙරටු කරුණු සොයනවා.", "target": "මම ඉගෙනීමේ පෙරටු කරුණු සොයමි."},
    {"input": "ඔවුන් විවරණයක් ලබා දෙයි.", "target": "ඔවුන් විවරණයක් ලබා දෙති."},
    {"input": "අපි ආහාර සූදානම් කරනවා.", "target": "අපි ආහාර සූදානම් කරමු."},
    {"input": "මම මිතුරන් හමුවීමට යනවා.", "target": "මම මිතුරන් හමුවීමට යමි."},
    {"input": "අපි සින්දු ගායනා කරනවා.", "target": "අපි සින්දු ගායනා කරමු."},
    {"input": "ඔහු කතා කරති.", "target": "ඔහු කතා කරයි."},
    {"input": "මම කාර්යභාරය ඉටු කරනවා.", "target": "මම කාර්යභාරය ඉටු කරමි."},
    {"input": "අපි වැඩකරු කාමරයට කැඳවනවා.", "target": "අපි වැඩකරු කාමරයට කැඳවමු."},
    {"input": "ඔවුන් වාර්තාවක් සකස් කරනවා.", "target": "ඔවුන් වාර්තාවක් සකස් කරති."},
    {"input": "මම වාර්තාව ලබා ගන්නවා.", "target": "මම වාර්තාව ලබා ගනිමි."},
    {"input": "අපි විභාගයට සූදානම් වෙනවා.", "target": "අපි විභාගයට සූදානම් වන්නෙමු."},
    {"input": "ඔවුන් පොත් ලබා ගනියි.", "target": "ඔවුන් පොත් ලබා ගනිති."},
    {"input": "මම නව මුහුණුවරක් සොයනවා.", "target": "මම නව මුහුණුවරක් සොයමි."},
    {"input": "අපි මිතුරන් එක්ක කතා කරනවා.", "target": "අපි මිතුරන් එක්ක කතා කරමු."},
    {"input": "ඔහු ගමන ආරම්භ කරති.", "target": "ඔහු ගමන ආරම්භ කරයි."},
    {"input": "මම පිටව යනවා.", "target": "මම පිටව යමි."},
    {"input": "අපි ගෙදර යනවා.", "target": "අපි ගෙදර යමු."},
    {"input": "ඔවුන් වැඩක් සකසන්නේ ය.", "target": "ඔවුන් වැඩක් සකසති."},
    {"input": "මම පොත් කියවීම ආරම්භ කරනවා.", "target": "මම පොත් කියවීම ආරම්භ කරමි."},
    {"input": "අපි අඩවියේ සිටිනවා.", "target": "අපි අඩවියේ සිටිමු."},
    {"input": "ඔහු ගෙදරට යන්න සූදානම් වූවෝ ය.", "target": "ඔහු ගෙදරට යන්න සූදානම් වේ."},
    {"input": "මම වැඩකට අත ගහනවා.", "target": "මම වැඩකට අත ගහමි."},
    {"input": "අපි නව ගැලරිය බලනවා.", "target": "අපි නව ගැලරිය බලමු."},
    {"input": "ඔවුන් මිතුරන් එක්ක විවේක ගන්නේ ය.", "target": "ඔවුන් මිතුරන් එක්ක විවේක ගන්නෝ ය."},
    {"input": "මම පරිගණකය අරඹනවා.", "target": "මම පරිගණකය අරඹමි."},
    {"input": "අපි ගීතයක් සකස් කරනවා.", "target": "අපි ගීතයක් සකස් කරමු."},
    {"input": "ඔහු වාර්තාවක් නිර්මාණය කරති.", "target": "ඔහු වාර්තාවක් නිර්මාණය කරයි."},
    {"input": "අපි  උදෑසන රැස්වීමට සහභාගී වෙනවා.", "target": "අපි උදෑසන රැස්වීමට සහභාගී වන්නෙමු."},
    {"input": "මම මිතුරන්ට පණිවුඩයක් යවනවා.", "target": "මම මිතුරන්ට පණිවුඩයක් යවමි."},
    {"input": "ඔවුන් උත්සවයක් සංවිධාන කරන්නේ ය.", "target": "ඔවුන් උත්සවයක් සංවිධාන කරන්නෝ ය."},
    {"input": "අපි වාදනයක් අසන්න යනවා.", "target": "අපි වාදනයක් අසන්න යමු."},
    {"input": "මම නිවස්සෙහි  උත්සවයක් සූදානම් කරනවා.", "target": "මම නිවස්සෙහි උත්සවයක් කරමි."},
    {"input": "ඔවුන් විනෝද ජනක සෙල්ලම් කරන්නේ ය.", "target": "ඔවුන් විනෝද ජනක සෙල්ලම් කරන්නෝ ය."},
    {"input": "අපි පොත් පදනමක් සොයනවා.", "target": "අපි පොත් පදනමක් සොයමු."},
    {"input": "මම ගිණුම් පරීක්ෂාවක් කරනවා.", "target": "මම ගිණුම් පරීක්ෂාවක් කරමි."},
    {"input": "ඔහු උසස් උත්සවයක් සමරති.", "target": "ඔහු උසස් උත්සවයක් සමරයි."},
    {"input": "අපි හොඳම විකල්පය තෝරා ගන්නවා.", "target": "අපි හොඳම විකල්පය තෝරා ගනිමු."},
    {"input": "මම ගීතයක් අසන්න සූදානම් වෙනවා.", "target": "මම ගීතයක් අසන්න සූදානම් වෙමි."},
    {"input": "ඔහු ගෙදරට ආපසු යති.", "target": "ඔහු ගෙදරට ආපසු යයි."},
    {"input": "අපි පල්ලියේ ගමනට සහභාගී වෙනවා.", "target": "අපි පල්ලියේ ගමනට සහභාගී වෙමු."},
    {"input": "ඔවුන් පරිගණකය අලුත් කරනේ ය.", "target": "ඔවුන් පරිගණකය අලුත් කරන්නෝ ය."},
    {"input": "මම නව සංකල්පයක් හදනවා.", "target": "මම නව සංකල්පයක් හදමි."},
    {"input": "අපි වර්ගීකරණය සොයනවා.", "target": "අපි වර්ගීකරණය සොයමු."},
    {"input": "ඔහු මිතුරෙකුට උපහාර කරති.", "target": "ඔහු මිතුරෙකුට උපහාර කරයි."},
    {"input": "අපි ජීවන මාර්ගයක් සොයනවා.", "target": "අපි ජීවන මාර්ගයක් සොයමු."},
    {"input": "මම ගොඩනැගිල්ලක් පිළිබඳ විග්‍රහ කරනවා.", "target": "මම ගොඩනැගිල්ලක් පිළිබඳ විග්‍රහ කරමි."},
    {"input": "ඔවුන් උත්සවය  අරඹනවා.", "target": "ඔවුන් උත්සවය  අරඹති."},
    {"input": "අපි මිතුරන් සමඟ විවේක ගන්නවා.", "target": "අපි මිතුරන් සමඟ විවේක ගනිමු."},
    {"input": "මම කාර්යය සම්පූර්ණ කරනවා.", "target": "මම කාර්යය සම්පූර්ණ කරමි."},
    {"input": "ඔහු කාර්යය නිම කරති.", "target": "ඔහු කාර්යය නිම කරයි."},
    {"input": "අපි ගැටලුවට පිළිතුරු සොයනවා.", "target": "අපි ගැටලුවට පිළිතුරු සොයමු."},
    {"input": "මම පණිවිඩයක් ලිවීම ආරම්භ කරනවා.", "target": "මම පණිවිඩයක් ලිවීම ආරම්භ කරමි."},
    {"input": "ඔවුන්  බයවෙයි.", "target": "ඔවුන් බයවෙති."},
    {"input": "අපි පාසලට පිටත් වෙනවා.", "target": "අපි පාසලට පිටත් වෙමු."},
    {"input": "මම දුරකථනය සකස් කරනවා.", "target": "මම දුරකථනය සකස් කරමි."},
    {"input": "ඔහු කාර්යාලයට යති.", "target": "ඔහු කාර්යාලයට යයි."},
    {"input": "අපි උත්සවය නරඹනවා.", "target": "අපි උත්සවය නරඹමු."},
    {"input": "මම නව යෝජනාවක් ඉදිරිපත් කරනවා.", "target": "මම නව යෝජනාවක් ඉදිරිපත් කරමි."},
    {"input": "ඔවුන්  අක්කව සූදානම් කරයි.", "target": "ඔවුන් අක්කව සූදානම් කරති."},
    {"input": "අපි විවේකයක් ගන්නවා.", "target": "අපි විවේකයක් ගනිමු."},
    {"input": "මම වැඩකට මූලිකත්වය ගන්නවා.", "target": "මම වැඩකට මූලිකත්වය ගනිමි."},
    {"input": "මම පරීකෂණයට ගිහින් නදි එක්ක පන්සල් යනවා. අපි රීට පස්සෙ ගවේශණය කරනවා මොකක්ද අවුල කියලා.", "target": "මම පරීකෂණයට ගිහින් නදි එක්ක පන්සල් යමි. අපි රීට පස්සෙ ගවේශනය කරමු මොකක්ද අවුල කියලා."},
    {"input": "මම සොයා ගන්නවා සහ අලුත්ම අදහසක් හිතනවා. අපි ඒවා බලාපොරොත්තු වෙනවා.", "target": "මම සොයා ගන්නෙමි සහ අලුත්ම අදහසක් හිතනෙමි. අපි ඒවා බලාපොරොත්තු වෙමු."},
    {"input": "මම නවීන තාක්ෂණය පරික්ෂා කරනවා. එතනින් ගවේශණය  පටන් ගන්නවා.", "target": "මම නවීන තාක්ෂණය පරික්ෂා කරන්නෙමි. එතනින් ගවේශනය පටන් ගන්නෙමි."},
    {"input": "අපි අනාගතය ගැන කතා කරනවා. නමුත් මම විශ්වාස කරනවා.", "target": "අපි අනාගතය ගැන කතා කරමු. නමුත් මම විශ්වාස කරෙමි."},
    {"input": "මම සාර්ථක වෙන්න හොඳ මාර්ගයක් හොයා ගන්නවා. අපි එකට වැඩ කරනවා.", "target": "මම සාර්ථක වෙන්න හොඳ මාර්ගයක් හොයා ගන්නෙමි. අපි එකට වැඩ කරමු."},
    {"input": "මම මගේ පර්යේෂණය දක්වා ජයග්‍රහණයක් ලබනවා.", "target": "මම මගේ පර්යේෂණය දක්වා ජයග්‍රහණයක් ලබන්නෙමි."},
    {"input": "ඔබ මට සහාය දෙනවා. මට කතා කරනවා.", "target": "ඔබ මට සහාය දෙයි. මට කතා කරනෙමි."},
    {"input": "අපි යනවා සහ ගවේෂණය කරනවා. අපි පරීක්ෂණය සම්පූර්ණ කරනවා.", "target": "අපි යමු සහ ගවේෂණය කරමු. අපි පරීක්ෂණය සම්පූර්ණ කරමු."},
    {"input": "මම කඩදාසි සකස් කරනවා. එහි සියලුම විස්තර ඉදිරිපත් කරනවා.", "target": "මම කඩදාසි සකස් කරනෙමි. එහි සියලුම විස්තර ඉදිරිපත් කරෙමි."},
    {"input": "ඔබට යොමු කරනවා. නමුත් මට මගේ දත්ත එකතු කරන්න ඉඩ නැතිව තිබෙනවා.", "target": "ඔබට යොමු කරයි. නමුත් මට මගේ දත්ත එකතු කරන්න ඉඩ නැතිව තිබෙමි."},
    {"input": "මම විශ්ලේෂණය කරනවා සහ දෝෂ නිවැරදි කරනු ලබනවා.", "target": "මම විශ්ලේෂණය කරෙමි සහ දෝෂ නිවැරදි කරෙමි."},
    {"input": "අපි කීප දිනක්ම නව තාක්ෂණය භාවිතා කලා. මට ඒවා සාර්ථකව ක්‍රියාත්මක කරන්න පුළුවන් වුනා.", "target": "අපි කීප  දිනක්ම නව තාක්ෂණය භාවිතා කලෙමු. මට ඒවා සාර්ථකව ක්‍රියාත්මක කරන්න පුළුවන් වුනෙමි."},
    {"input": "මම ප්‍රායෝගික කටයුත්තක් ආරම්භ කරනවා. එය නව යෝජනා සහිතව ඇත.", "target": "මම ප්‍රායෝගික කටයුත්තක් ආරම්භ කරෙමි. එය නව යෝජනා සහිතව ඇත."},
    {"input": "අපි රීට පස්සේ පරීක්ෂණ සම්පූර්ණ කරනවා.", "target": "අපි රීට පස්සෙ පරීක්ෂණ සම්පූර්ණ කරමු."},
    {"input": "මම විභාග පසු  කලෙමු.", "target": "මම විභාග පසු  කලෙමි."},
    {"input": "මම ඉක්මනින් ගිහින් කාර්යය ඉක්මනින් කලෙමු.", "target": "මම ඉක්මනින් ගිහින් කාර්යය ඉක්මනින් කලෙමි."},
    {"input": "අපි කාර්යය සිදු කරනවා, නමුත් එය අවසන් කරන්නේ කවදාද?", "target": "අපි කාර්යය සිදු කරමු, නමුත් එය අවසන් කරන්නේ කවදාද?"},
    {"input": "මම පරීක්ෂණයට ගිහින් නදි එක්ක පන්සල් යනවා. අපි රීට පස්සෙ ගවේශණය කරනවා මොකක්ද අවුල කියලා. මම නවීන තාක්ෂණය පරික්ෂා කරනවා. එතනින් ගවේශණය පටන් ගන්නවා.",
     "target": "මම පරීක්ෂණයට ගිහින් නදි එක්ක පන්සල් යමි. අපි රීට පස්සෙ ගවේශනය කරමු මොකක්ද අවුල කියලා. මම නවීන තාක්ෂණය පරික්ෂා කරන්නෙමි. එතනින් ගවේශනය පටන් ගන්නෙමි."},
    {"input": "මම සොයා ගන්නවා සහ අලුත්ම අදහසක් හිතනවා. අපි ඒවා බලාපොරොත්තු වෙනවා. අපි අනාගතය ගැන කතා කරනවා. නමුත් මම විශ්වාස කරනවා.",
     "target": "මම සොයා ගන්නෙමි සහ අලුත්ම අදහසක් හිතනෙමි. අපි ඒවා බලාපොරොත්තු වෙමු. අපි අනාගතය ගැන කතා කරමු. නමුත් මම විශ්වාස කරෙමි."},
    {"input": "අපි කාර්යය සිදු කරනවා, නමුත් එය අවසන් කරන්නේ කවදාද? මම කඩදාසි සකස් කරනවා. එහි සියලුම විස්තර ඉදිරිපත් කරනවා.",
     "target": "අපි කාර්යය සිදු කරමු, නමුත් එය අවසන් කරන්නේ කවදාද? මම කඩදාසි සකස් කරනෙමි. එහි සියලුම විස්තර ඉදිරිපත් කරෙමි."},
    {"input": "මම කතාව පවත්වනවා. අලුත්ම අදහසක් සැලසුම් කරනවා.","target": "මම කතාව පවත්තන්නෙමි. අලුත්ම අදහසක් සැලසුම් කරෙමි."},

    {"input": "අපි සාර්ථක වෙනවා. මම බලාපොරොත්තු වෙනවා.","target": "අපි සාර්ථක වෙමු. මම බලාපොරොත්තු වෙමි."},

    {"input": "මම දත්ත එකතු කරනවා. ඒවා විශ්ලේෂණය කරනවා.","target": "මම දත්ත එකතු කරෙමි. ඒවා විශ්ලේෂණය කරෙමි."},

    {"input": "අපි නිවැරදි ක්‍රමයක් නිර්මාණය කරනවා. පසුව එය පරීක්ෂා කරනවා.","target": "අපි නිවැරදි ක්‍රමයක් නිර්මාණය කරමු. පසුව එය පරීක්ෂා කරමු."},

    {"input": "මම සටහන් තබනවා. එය පසුව සමාලෝචනය කරනවා.","target": "මම සටහන් තබනෙමි. එය පසුව සමාලෝචනය කරෙමි."},

    {"input": "අපි වැඩ පවත්වනවා, නමුත් එය අවසන් කරන්නේ කවදාද?","target": "අපි වැඩ පවත්වෙමු. නමුත් එය අවසන් කරන්නේ කවදාද?"},

    {"input": "මම බලාපොරොත්තු වෙනවා. මට ඉතාමත් ඉක්මනින් කරන්න අවශ්‍යයි.","target": "මම බලාපොරොත්තු වෙමි. මට ඉතාමත්  ඉක්මනින් කරන්න අවශ්‍යෙමි."},

    {"input": "අපි පනස්වෙනි දිනයේදී වැඩ කිරීමෙන් පසුව අවසන් කරනවා. මට අලුත්ම අදහසක් තිබෙනවා.","target": "අපි පනස්වෙනි දිනයේදී වැඩ කිරීමෙන් පසුව අවසන් කරමු. මට අලුත්ම අදහසක් තිබෙමි."},

    {"input": "මම මට සැලසුම් කරනවා. එය නව වැඩක් වශයෙන් වේ.","target": "මම මට සැලසුම් කරෙමි. එය නව වැඩක් වශයෙන් වේ."},

    {"input": "අපි ආරක්ෂාව පරීක්ෂා කරනවා. එය සාර්ථකව අගය කරනවා.","target": "අපි ආරක්ෂාව පරීක්ෂා කරමු. එය සාර්ථකව අගය කරනවා."},

    {"input": "මම දියුණු කරන්නේ මගේ පර්යේෂණය. මට කුමක් කරන්න අවශ්‍යයි.","target": "මම දියුණු කරෙමි මගේ පර්යේෂණය. මට කුමක් කරන්න අවශ්‍යෙමි."},

    {"input": "අපි කුමක් හෝ ගවේෂණය කරන්නෙමු. එය හොඳම ක්‍රමය වේ.","target": "අපි කුමක් හෝ ගවේෂණය කරමු. එය හොඳම ක්‍රමය වේ."},

    {"input": "මම ගවේෂණය කරනවා. ඒවා පරික්ෂා කරනවා.","target": "මම ගවේෂණය කරෙමි. ඒවා පරික්ෂා කරනවා."},

    {"input": "මම බොහෝ දුරට පටන් ගන්නවා. එය පසුව සාර්ථකව ක්‍රියාත්මක කරනවා.","target": "මම බොහෝ දුරට පටන් ගන්නෙමි. එය පසුව සාර්ථකව ක්‍රියාත්මක කරනවා."},

    {"input": "මට වැඩ කිරීමේ අවශ්‍යතාවක් පවතිනවා. මට බලාපොරොත්තු තියෙනවා.","target": "මට වැඩ කිරීමේ අවශ්‍යතාවක් පවතිනෙමි. මට බලාපොරොත්තු තියෙමි."},

    {"input": "අපි පසුව යොමු කරනවා. එය මට අවශ්‍යයි.","target": "අපි පසුව යොමු කරමු. එය මට අවශ්‍යයි."},

    {"input": "මම වැටලීමකට යනවා. එය වැඩි දුරට හොඳ වේ.","target": "මම වැටලීමකට යමි. එය වැඩි දුරට හොඳ වේ."},

    {"input": "මම කාර්යය සම්පූර්ණ කරනවා. අපි එකට යනවා.","target": "මම කාර්යය සම්පූර්ණ කරෙමි. අපි එකට යමු."},

    {"input": "අපි මහා පාර්ලිමේන්තුවට යනවා. මට විශ්වාසයක් පවතී.","target": "අපි මහා පාර්ලිමේන්තුවට යමු. මට විශ්වාසයක් පවතිමි."},

    {"input": "මම එය සලසනවා. එය මගේ කාර්යය.","target": "මම එය සලසනෙමි. එය මගේ කාර්යය."},

    {"input": "අපි පසුව සංග්‍රහ කරනවා. එය අපට ප්‍රයෝජනවත් වේ.","target": "අපි පසුව සංග්‍රහ කරමු. එය අපට ප්‍රයෝජනවත් වේ."},

    {"input": "මට දැනට ප්‍රතිලාභ නැත. මම කුමක් හෝ කිරීමට යනවා.","target": "මට දැනට ප්‍රතිලාභ නැත. මම කුමක් හෝ කිරීමට යමි."},

    {"input": "අපි ගවේශණය කරනවා. මට තවත් තොරතුරු අවශ්‍යයි.","target": "අපි ගවේශනය කරමු. මට තවත් තොරතුරු අවශයෙමි."},

    {"input": "මට පනස්වෙනි වැඩෙත් නිමවන්නේ යයි සිතනවා. අපි සාර්ථකව අවසන් කරනවා.","target": "මට පනස්වෙනි වැඩෙත් නිමවන්නේ යයි සිතනෙමි. අපි සාර්ථකව අවසන් කරමු."},
  {"input": "මම ගෙදර ගියෙමු.", "target": "මම ගෙදර ගියෙමි."},
  {"input": "මම ගෙදර ගියේ ය.", "target": "මම ගෙදර ගියෙමි."},
  {"input": "මම ගෙදර ගියෝ ය.", "target": "මමගෙදර ගියෙමි."},
  {"input": "මම ගෙදර ගියා.", "target": "මම ගෙදර ගියෙමි."},
  {"input": "මම ගෙදර යනවා.", "target": "මම ගෙදර යමි."},
  {"input": "මම ගෙදර යන්නෙමි.", "target": "මම ගෙදර යන්නෙමි."},

  {"input": "මම බත් කන්නෙමු.", "target": "මම බත් කන්නෙමි."},
  {"input": "මම බත් කන්නේ ය.", "target": "මම බත් කෑවෙමි."},
  {"input": "මම බත් කන්නෝ ය.", "target": "මම බත් කෑවෙමි."},
  {"input": "මම බත් කෑවා.", "target": "මම බත් කෑවෙමි."},
  {"input": "මම බත් කනවා.", "target": "මම බත් කමි."},

  {"input": "මම කැලේ නිදාගන්නෙමු.", "target": "මම කැලේ නිදාගන්නෙමි."},
  {"input": "මම කැලේ නිදාගත්තේ ය.", "target": "මම කැලේ නිදාගත්තෙමි."},
  {"input": "මම කැලේ නිදාගතෝ ය.", "target": "මම කැලේ නිදාගත්තෙමි."},
  {"input": "මම කැලේ නිදාගත්තා.", "target": "මම කැලේ නිදාගත්තෙමි."},
  {"input": "මම කැලේ නිදා ගන්නවා.", "target": "මම කැලේ නිදාගනිමි."},

  {"input": "මම බත් උයන්නෙමු.", "target": "මම බත් උයන්නෙමි."},
  {"input": "මම බත් උයන්නේ ය.", "target": "මම බත් ඉව්වෙමි."},
  {"input": "මම බත් උයන්නෝ ය.", "target": "මම බත් ඉව්වෙමි."},
  {"input": "මම බත් ඉව්වා.", "target": "මම බත් ඉව්වෙමි."},
  {"input": "මම බත් උයනවා.", "target": "මම බත් උයමි."},

  {"input": "මම ගස් නගින්නෙමු.", "target": "මම ගස් නගින්නෙමි."},
  {"input": "මම ගස් නගින්නේ ය.", "target": "මම ගස් නැග්ගෙමි."},
  {"input": "මම ගස් නගින්නෝ ය.", "target": "මම ගස් නැග්ගෙමි."},
  {"input": "මම ගස් නැග්ගා.", "target": "මම ගස් නැග්ගෙමි."},
  {"input": "මම ගස් නගිනවා.", "target": "මම ගස් නගිමි."},

  {"input": "මම පලතුරු කන්නෙමු.", "target": "මම පලතුරු කන්නෙමි."},
  {"input": "මම පලතුරු කන්නේ ය.", "target": "මම පලතුරු කෑවෙමි."},
  {"input": "මම පලතුරු කන්නෝ ය.", "target": "මම පලතුරු කෑවෙමි."},
  {"input": "මම පලතුරු කෑවා.", "target": "මම පලතුරු කෑවෙමි."},
  {"input": "මම පලතුරු කනවා.", "target": "මම පලතුරු කමි."},

  {"input": "මම මල් කඩන්නෙමු.", "target": "මම මල් කඩන්නෙමි."},
  {"input": "මම මල් කන්නේ ය.", "target": "මම මල් කැඩුවෙමි."},
  {"input": "මම මල් කන්නෝ ය.", "target": "මම මල් කැඩුවෙමි."},
  {"input": "මම මල් කැඩුවා.", "target": "මම මල් කැඩුවෙමි."},
  {"input": "මම මල් කඩනවා.", "target": "මම මල් කඩමි."},

  {"input": "මම රබන් ගසන්නෙමු.", "target": "මම රබන් ගසන්නෙමි."},
  {"input": "මම රබන් ගසන්නේ ය.", "target": "මම රබන් ගැසුවෙමි."},
  {"input": "මම රබන් ගසන්නෝ ය.", "target": "මම රබන් ගැසුවෙමි."},
  {"input": "මම රබන් ගැසුවා.", "target": "මම රබන් ගැසුවෙමි."},
  {"input": "මම රබන් ගහනවා.", "target": "මම රබන් ගසමි."},

  {"input": "මම බන අසන්නෙමු.", "target": "මම බන අසන්නෙමි."},
  {"input": "මම බන අසන්නේ ය.", "target": "මම බන ඇසුවෙමි."},
  {"input": "මම බන අසන්නෝ ය.", "target": "මම බන ඇසුවෙමි."},
  {"input": "මම බන ගැසුවා.", "target": "මම බන ඇසුවෙමි."},
  {"input": "මම බන ගහනවා.", "target": "මම බන අසමි."},

  {"input": "මම ආහාර පිසින්නෙමු.", "target": "මම ආහාර පිසින්නෙමි."},
  {"input": "මම ආහාර පිසින්නේ ය.", "target": "මම ආහාර පිසුවෙමි."},
  {"input": "මම ආහාර පිසින්නෝ ය.", "target": "මම ආහාර පිසුවෙමි."},
  {"input": "මම ආහාර පිසුවා.", "target": "මම ආහාර පිසුවෙමි."},
  {"input": "මම ආහාර පිසිනවා.", "target": "මම ආහාර පිසිමි."},

  {"input": "මම බෝඩිමට යන්නෙමු.", "target": "මම බෝඩිමට යන්නෙමි."},
  {"input": "මම බෝඩිමට ගියේ ය.", "target": "මම බෝඩිමට ගියෙමි."},
  {"input": "මම බෝඩිමට ගියෝ ය.", "target": "මම බෝඩිමට ගියෙමි."},
  {"input": "මම බෝඩිමට ගියා.", "target": "මම බෝඩිමට ගියෙමි."},
  {"input": "මම බෝඩිමට යනවා.", "target": "මම බෝඩිමට යමි."},

  {"input": "මම පාසලට යන්නෙමු.", "target": "මම පාසලට යන්නෙමි."},
  {"input": "මම පාසලට ගියේ ය.", "target": "මම පාසලට ගියෙමි."},
  {"input": "මම පාසලට ගියෝ ය.", "target": "මම පාසලට ගියෙමි."},
  {"input": "මම පාසලට ගියා.", "target": "මම පාසලට ගියෙමි."},
  {"input": "මම පාසලට යනවා.", "target": "මම පාසලට යමි."},

  {"input": "මම චිත්‍ර අදින්නෙමු.", "target": "මම චිත්‍ර අදින්නෙමි."},
  {"input": "මම චිත්‍ර ඇන්දේ ය.", "target": "මම චිත්‍ර ඇන්දෙමි."},
  {"input": "මම චිත්‍ර ඇන්දෝ ය.", "target": "මම චිත්‍ර ඇන්දෙමි."},
  {"input": "මම චිත්‍ර ඇන්දා.", "target": "මම චිත්‍ර ඇන්දෙමි."},
  {"input": "මම චිත්‍ර අදිනවා.", "target": "මම චිත්‍ර අදිමි."},

  {"input": "මම පන්තියට යන්නෙමු.", "target": "මම පන්තියට යන්නෙමි."},
  {"input": "මම පන්තියට ගියේ ය.", "target": "මම පන්තියට ගියෙමි."},
  {"input": "මම පන්තියට ගියෝ ය.", "target": "මම පන්තියට ගියෙමි."},
  {"input": "මම පන්තියට ගියා.", "target": "මම පන්තියට ගියෙමි."},
  {"input": "මම පන්තියට යනවා.", "target": "මම පන්තියට යමි."},

  {"input": "මම සාකච්ඡාව කරන්නෙමු.", "target": "මම සාකච්ඡාව කරන්නෙමි."},
  {"input": "මම සාකච්ඡාව කලේ ය.", "target": "මම සාකච්ඡාව කලෙමි."},
  {"input": "මම සාකච්ඡාව කලෝ ය.", "target": "මම සාකච්ඡාව කලෙමි."},
  {"input": "මම සාකච්ඡාව කලා.", "target": "මම සාකච්ඡාව කලෙමි."},
  {"input": "මම සාකච්ඡාව කරනවා.", "target": "මම සාකච්ඡාව කරමි."},

  {"input": "මම ගමනාන්තයට ලගා වෙන්නෙමු.", "target": "මම ගමනාන්තයට ලගා වෙන්නෙමි."},
  {"input": "මම ගමනාන්තයට ලගා වුනේ ය.", "target": "මම ගමනාන්තයට ලගා වුනෙමි."},
  {"input": "මම ගමනාන්තයට ලගා වුනෝ ය.", "target": "මම ගමනාන්තයට ලගා වුනෙමි."},
  {"input": "මම ගමනාන්තයට ලගා වුනා.", "target": "මම ගමනාන්තයට ලගා වුනෙමි."},
  {"input": "මම ගමනාන්තයට ලගා වෙනවා.", "target": "මම ගමනාන්තයට ලගා වෙමි."},

  {"input": "මම මල් පූජා කරන්නෙමු.", "target": "මම මල් පූජා කරන්නෙමි."},
  {"input": "මම මල් පූජා කලේ ය.", "target": "මම මල් පූජා කලෙමි."},
  {"input": "මම මල් පූජා කලෝ ය.", "target": "මම මල් පූජා කලෙමි."},
  {"input": "මම මල් පූජා කලා.", "target": "මම මල් පූජා කලෙමි."},
  {"input": "මම මල් පූජා කරනවා.", "target": "මම මල් පූජා කරමි."},

  {"input": "මම ගමන ආරම්භ කරන්නෙමු.", "target": "මම ගමන ආරම්භ කරන්නෙමි."},
  {"input": "මම ගමන ආරම්භ කලේ ය.", "target": "මම ගමන ආරම්භ කලෙමි."},
  {"input": "මම ගමන ආරම්භ කලෝ ය.", "target": "මම ගමන ආරම්භ කලෙමි."},
  {"input": "මම ගමන ආරම්භ කලා.", "target": "මම ගමන ආරම්භ කලෙමි."},
  {"input": "මම ගමන ආරම්භ කරනවා.", "target": "මම ගමන ආරම්භ කරමි."},

  {"input": "මම කුටිය අලුත් කරන්නෙමු.", "target": "මම කුටිය අලුත් කරන්නෙමි."},
  {"input": "මම කුටිය අලුත් කලේ ය.", "target": "මම කුටිය අලුත් කලෙමි."},
  {"input": "මම කුටිය අලුත් කලෝ ය.", "target": "මම කුටිය අලුත් කලෙමි."},
  {"input": "මම කුටිය අලුත් කලා.", "target": "මම කුටිය අලුත් කලෙමි."},
  {"input": "මම කුටිය අලුත් කරනවා.", "target": "මම කුටිය අලුත් කරමි."},

  {"input": "මම දුරකතනය පාවිච්චි කරන්නෙමු.", "target": "මම දුරකතනය පාවිච්චි කරන්නෙමි."},
  {"input": "මම දුරකතනය පාවිච්චි කරන්නේ ය.", "target": "මම දුරකතනය පාවිච්චි කලෙමි."},
  {"input": "මම දුරකතනය පාවිච්චි කරන්නෝ ය.", "target": "මම දුරකතනය පාවිච්චි කලෙමි."},
  {"input": "මම දුරකතනය පාවිච්චි කෙරුවා.", "target": "මම දුරකතනය පාවිච්චි කලෙමි."},
  {"input": "මම දුරකතනය පාවිච්චි කරනවා.", "target": "මම දුරකතනය පාවිච්චි කරමි."},

  {"input": "මම සෙල්ලම් කරන්න යන්නෙමු.", "target": "මම සෙල්ලම් කරන්න යන්නෙමි."},
  {"input": "මම සෙල්ලම් කරන්න යන්නේ ය.", "target": "මම සෙල්ලම් කරන්න ගියෙමි."},
  {"input": "මම සෙල්ලම් කරන්න යන්නෝ ය.", "target": "මම සෙල්ලම් කරන්න ගියෙමි."},
  {"input": "මම සෙල්ලම් කරන්න ගියා.", "target": "මම සෙල්ලම් කරන්න ගියෙමි."},
  {"input": "මම සෙල්ලම් කරන්න යනවා.", "target": "මම සෙල්ලම් කරන්න යමි."},

  {"input": "මම පොත් පාඩම් කරන්නෙමු.", "target": "මම පොත් පාඩම් කරන්නෙමි."},
  {"input": "මම පොත් පාඩම් කරන්නේ ය.", "target": "මම පොත් පාඩම් කලෙමි."},
  {"input": "මම පොත් පාඩම් කරන්නෝ ය.", "target": "මම පොත් පාඩම් කලෙමි."},
  {"input": "මම පොත් පාඩම් කලා.", "target": "මම පොත් පාඩම් කලෙමි."},
  {"input": "මම පොත් පාඩම් කරනවා.", "target": "මම පොත් පාඩම් කරමි."},

  {"input": "මම මිදුල අතු ගාන්නෙමු.", "target": "මම මිදුල අතු ගාන්නෙමි."},
  {"input": "මම මිදුල අතු ගාන්නේ ය.", "target": "මම මිදුල අතු ගෑවෙමි."},
  {"input": "මම මිදුල අතු ගාන්නෝ ය.", "target": "මම මිදුල අතු ගෑවෙමි."},
  {"input": "මම මිදුල අතු ගෑවා.", "target": "මම මිදුල අතු ගෑවෙමි."},
  {"input": "මම මිදුල අතු ගානවා.", "target": "මම මිදුල අතු ගාමි."},

  {"input": "මම අලුත් අවුරුද්ද සමරන්නෙමු.", "target": "මම අලුත් අවුරුද්ද සමරන්නෙමි."},
  {"input": "මම අලුත් අවුරුද්ද සමරන්නේ ය.", "target": "මම අලුත් අවුරුද්ද සැමරුවෙමි."},
  {"input": "මම අලුත් අවුරුද්ද සමරන්නෝ ය.", "target": "මම අලුත් අවුරුද්ද සැමරුවෙමි."},
  {"input": "මම අලුත් අවුරුද්ද සැමරුවා.", "target": "මම අලුත් අවුරුද්ද සැමරුවෙමි."},
  {"input": "මම අලුත් අවුරුද්ද සමරනවා.", "target": "මම අලුත් අවුරුද්ද සමරමි."},

  {"input": "මම පරිසරය පිරිසිදු කරන්නෙමු.", "target": "මම පරිසරය පිරිසිදු කරන්නෙමි."},
  {"input": "මම පරිසරය පිරිසිදු කලේ ය.", "target": "මම පරිසරය පිරිසිදු කලෙමි."},
  {"input": "මම පරිසරය පිරිසිදු කලෝ ය.", "target": "මම පරිසරය පිරිසිදු කලෙමි."},
  {"input": "මම පරිසරය පිරිසිදු කලා.", "target": "මම පරිසරය පිරිසිදු කලෙමි."},
  {"input": "මම පරිසරය පිරිසිදු කරනවා.", "target": "මම පරිසරය පිරිසිදු කලෙමි."},

  {"input": "මම අම්මව බලන්න  යන්නෙමු.", "target": "මම අම්මව බලන්න යන්නෙමි."},
  {"input": "මම අම්මව බලන්න යන්නේ ය.", "target": "මම අම්මව බලන්න ගියෙමි."},
  {"input": "මම අම්මව බලන්න යන්නෝ ය.", "target": "මම අම්මව බලන්න ගියෙමි."},
  {"input": "මම අම්මව බලන්න ගියා.", "target": "මම අම්මව බලන්න ගියෙමි."},
  {"input": "මම අම්මව බලන්න යනවා.", "target": "මම අම්මව බලන්න ගියෙමි."},

  {"input": "මම සතුරන්ගෙන් පලිගන්න යන්නෙමු.", "target": "මම සතුරන්ගෙන් පලිගන්න යන්නෙමි."},
  {"input": "මම සතුරන්ගෙන් පලිගන්න යන්නේ ය.", "target": "මම සතුරන්ගෙන් පලිගන්න ගියෙමි."},
  {"input": "මම සතුරන්ගෙන් පලිගන්න යන්නෝ ය.", "target": "මම සතුරන්ගෙන් පලිගන්න ගියෙමි."},
  {"input": "මම සතුරන්ගෙන් පලිගන්න ගියා.", "target": "මම සතුරන්ගෙන් පලිගන්න ගියෙමි."},
  {"input": "මම සතුරන්ගෙන් පලිගන්න යනවා.", "target": "මම සතුරන්ගෙන් පලිගන්න යමි."},

  {"input": "මම කිසිවෙකුට කරදර නොකරන්නෙමු.", "target": "මම කිසිවෙකුට කරදර නොකරන්නෙමි."},
  {"input": "මම කිසිවෙකුට කරදර නොකරන්නේ ය.", "target": "මම කිසිවෙකුට කරදර නොකලෙමි."},
  {"input": "මම කිසිවෙකුට කරදර නොකරන්නෝ ය.", "target": "මම කිසිවෙකුට කරදර නොකලෙමි."},
  {"input": "මම කිසිවෙකුට කරදර නොකලා.", "target": "මම කිසිවෙකුට කරදර නොකලෙමි."},
  {"input": "මම කිසිවෙකුට කරදර නොකරනවා.", "target": "මම කිසිවෙකුට කරදර නොකරමි."},

  {"input": "මම කැලේ සංචාරය කරන්නෙමු.", "target": "මම කැලේ සංචාරය කරන්නෙමි."},
  {"input": "මම කැලේ සංචාරය කරන්නේ ය.", "target": "මම කැලේ සංචාරය කලෙමි."},
  {"input": "මම කැලේ සංචාරය කරන්නෝ ය.", "target": "මම කැලේ සංචාරය කලෙමි."},
  {"input": "මම කැලේ සංචාරය කලා.", "target": "මම කැලේ සංචාරය කලෙමි."},
  {"input": "මම කැලේ සංචාරය කරනවා.", "target": "මම කැලේ සංචාරය කලෙමි."},

  {"input": "මම දරුවව හොදින් බලා ගන්නෙමු.", "target": "මම දරුවව හොදින් බලා ගන්නෙමි."},
  {"input": "මම දරුවව හොදින් බලා ගන්නේ ය.", "target": "මම දරුවව හොදින් බලා ගත්තෙමි."},
  {"input": "මම දරුවව හොදින් බලා ගන්නෝ ය.", "target": "මම දරුවව හොදින් බලා ගත්තෙමි."},
  {"input": "මම දරුවව හොදින් බලා ගත්තා.", "target": "මම දරුවව හොදින් බලා ගත්තෙමි."},
  {"input": "මම දරුවව හොදින් බලා ගන්නවා.", "target": "මම දරුවව හොදින් බලා ගමි."},

  {"input": "අපි ගෙදර ගියෙමි.", "target": "අපි ගෙදර ගියෙමු."},
  {"input": "අපි ගෙදර ගියේ ය.", "target": "අපි ගෙදර ගියෙමු."},
  {"input": "අපි ගෙදර ගියෝ ය.", "target": "අපි ගෙදර ගියෙමු."},
  {"input": "අපි ගෙදර ගියා.", "target": "අපි ගෙදර ගියෙමු."},
  {"input": "අපි ගෙදර යනවා.", "target": "අපි ගෙදර යමු."},
  {"input": "අපි ගෙදර යන්නෙමි.", "target": "අපි ගෙදර යන්නෙමු."},

  {"input": "අපි බත් කන්නෙමි.", "target": "අපි බත් කන්නෙමු."},
  {"input": "අපි බත් කන්නේ ය.", "target": "අපි බත් කෑවෙමු."},
  {"input": "අපි බත් කන්නෝ ය.", "target": "අපි බත් කෑවෙමු."},
  {"input": "අපි බත් කෑවා.", "target": "අපි බත් කෑවෙමු."},
  {"input": "අපි බත් කනවා.", "target": "අපි බත් කමු."},

  {"input": "අපි කැලේ නිදාගන්නෙමි.", "target": "අපි කැලේ නිදාගන්නෙමු."},
  {"input": "අපි කැලේ නිදාගත්තේ ය.", "target": "අපි කැලේ නිදාගත්තෙමු."},
  {"input": "අපි කැලේ නිදාගතෝ ය.", "target": "අපි කැලේ නිදාගත්තෙමු."},
  {"input": "අපි කැලේ නිදාගත්තා.", "target": "අපි කැලේ නිදාගත්තෙමු."},
  {"input": "අපි කැලේ නිදා ගන්නවා.", "target": "අපි කැලේ නිදාගනිමු."},

  {"input": "අපි බත් උයන්නෙමි.", "target": "අපි බත් උයන්නෙමු."},
  {"input": "අපි බත් උයන්නේ ය.", "target": "අපි බත් ඉව්වෙමු."},
  {"input": "අපි බත් උයන්නෝ ය.", "target": "අපි බත් ඉව්වෙමු."},
  {"input": "අපි බත් ඉව්වා.", "target": "අපි බත් ඉව්වෙමු."},
  {"input": "අපි බත් උයනවා.", "target": "අපි බත් උයමු."},

  {"input": "අපි ගස් නගින්නෙමි.", "target": "අපි ගස් නගින්නෙමු."},
  {"input": "අපි ගස් නගින්නේ ය.", "target": "අපි ගස් නැග්ගෙමු."},
  {"input": "අපි ගස් නගින්නෝ ය.", "target": "අපි ගස් නැග්ගෙමු."},
  {"input": "අපි ගස් නැග්ගා.", "target": "අපි ගස් නැග්ගෙමු."},
  {"input": "අපි ගස් නගිනවා.", "target": "අපි ගස් නගිමු."},

  {"input": "අපි පලතුරු කන්නෙමි.", "target": "අපි පලතුරු කන්නෙමු."},
  {"input": "අපි පලතුරු කන්නේ ය.", "target": "අපි පලතුරු කෑවෙමු."},
  {"input": "අපි පලතුරු කන්නෝ ය.", "target": "අපි පලතුරු කෑවෙමු."},
  {"input": "අපි පලතුරු කෑවා.", "target": "අපි පලතුරු කෑවෙමු."},
  {"input": "අපි පලතුරු කනවා.", "target": "අපි පලතුරු කමු."},

  {"input": "අපි මල් කඩන්නෙමි.", "target": "අපි මල් කඩන්නෙමු."},
  {"input": "අපි මල් කන්නේ ය.", "target": "අපි මල් කැඩුවෙමු."},
  {"input": "අපි මල් කන්නෝ ය.", "target": "අපි මල් කැඩුවෙමු."},
  {"input": "අපි මල් කැඩුවා.", "target": "අපි මල් කැඩුවෙමු."},
  {"input": "අපි මල් කඩනවා.", "target": "අපි මල් කඩමු."},

{"input": "අපි රබන් ගසන්නෙමි.", "target": "අපි රබන් ගසන්නෙමු."},
{"input": "අපි රබන් ගසන්නේ ය.", "target": "අපි රබන් ගැසුවෙමු."},
{"input": "අපි රබන් ගසන්නෝ ය.", "target": "අපි රබන් ගැසුවෙමු."},
{"input": "අපි රබන් ගැසුවා.", "target": "අපි රබන් ගැසුවෙමු."},
{"input": "අපි රබන් ගහනවා.", "target": "අපි රබන් ගසමු."},

{"input": "අපි බන අසන්නෙමි.", "target": "අපි බන අසන්නෙමු."},
{"input": "අපි බන අසන්නේ ය.", "target": "අපි බන ඇසුවෙමු."},
{"input": "අපි බන අසන්නෝ ය.", "target": "අපි බන ඇසුවෙමු."},
{"input": "අපි බන ගැසුවා.", "target": "අපි බන ඇසුවෙමු."},
{"input": "අපි බන ගහනවා.", "target": "අපි බන අසමු."},

{"input": "අපි ආහාර පිසින්නෙමි.", "target": "අපි ආහාර පිසින්නෙමු."},
{"input": "අපි ආහාර පිසින්නේ ය.", "target": "අපි ආහාර පිසුවෙමු."},
{"input": "අපි ආහාර පිසින්නෝ ය.", "target": "අපි ආහාර පිසුවෙමු."},
{"input": "අපි ආහාර පිසුවා.", "target": "අපි ආහාර පිසුවෙමු."},
{"input": "අපි ආහාර පිසිනවා.", "target": "අපි ආහාර පිසිමු."},

{"input": "අපි බෝඩිමට යන්නෙමි.", "target": "අපි බෝඩිමට යන්නෙමු."},
{"input": "අපි බෝඩිමට ගියේ ය.", "target": "අපි බෝඩිමට ගියෙමු."},
{"input": "අපි බෝඩිමට ගියෝ ය.", "target": "අපි බෝඩිමට ගියෙමු."},
{"input": "අපි බෝඩිමට ගියා.", "target": "අපි බෝඩිමට ගියෙමු."},
{"input": "අපි බෝඩිමට යනවා.", "target": "අපි බෝඩිමට යමු."},

{"input": "අපි පාසලට යන්නෙමි.", "target": "අපි පාසලට යන්නෙමු."},
{"input": "අපි පාසලට ගියේ ය.", "target": "අපි පාසලට ගියෙමු."},
{"input": "අපි පාසලට ගියෝ ය.", "target": "අපි පාසලට ගියෙමු."},
{"input": "අපි පාසලට ගියා.", "target": "අපි පාසලට ගියෙමු."},
{"input": "අපි පාසලට යනවා.", "target": "අපි පාසලට යමු."},

{"input": "අපි චිත්‍ර අදින්නෙමි.", "target": "අපි චිත්‍ර අදින්නෙමු."},
{"input": "අපි චිත්‍ර ඇන්දේ ය.", "target": "අපි චිත්‍ර ඇන්දෙමු."},
{"input": "අපි චිත්‍ර ඇන්දෝ ය.", "target": "අපි චිත්‍ර ඇන්දෙමු."},
{"input": "අපි චිත්‍ර ඇන්දා.", "target": "අපි චිත්‍ර ඇන්දෙමු."},
{"input": "අපි චිත්‍ර අදිනවා.", "target": "අපි චිත්‍ර අදිමු."},

{"input": "අපි පන්තියට යන්නෙමි.", "target": "අපි පන්තියට යන්නෙමු."},
{"input": "අපි පන්තියට ගියේ ය.", "target": "අපි පන්තියට ගියෙමු."},
{"input": "අපි පන්තියට ගියෝ ය.", "target": "අපි පන්තියට ගියෙමු."},
{"input": "අපි පන්තියට ගියා.", "target": "අපි පන්තියට ගියෙමු."},
{"input": "අපි පන්තියට යනවා.", "target": "අපි පන්තියට යමු."},

{"input": "අපි සාකච්ඡාව කරන්නෙමි.", "target": "අපි සාකච්ඡාව කරන්නෙමු."},
{"input": "අපි සාකච්ඡාව කලේ ය.", "target": "අපි සාකච්ඡාව කලෙමු."},
{"input": "අපි සාකච්ඡාව කලෝ ය.", "target": "අපි සාකච්ඡාව කලෙමු."},
{"input": "අපි සාකච්ඡාව කලා.", "target": "අපි සාකච්ඡාව කලෙමු."},
{"input": "අපි සාකච්ඡාව කරනවා.", "target": "අපි සාකච්ඡාව කරමු."},

{"input": "අපි ගමනාන්තයට ලගා වෙන්නෙමි.", "target": "අපි ගමනාන්තයට ලගා වෙන්නෙමු."},
{"input": "අපි ගමනාන්තයට ලගා වුනේ ය.", "target": "අපි ගමනාන්තයට ලගා වුනෙමු."},
{"input": "අපි ගමනාන්තයට ලගා වුනෝ ය.", "target": "අපි ගමනාන්තයට ලගා වුනෙමු."},
{"input": "අපි ගමනාන්තයට ලගා වුනා.", "target": "අපි ගමනාන්තයට ලගා වුනෙමු."},
{"input": "අපි ගමනාන්තයට ලගා වෙනවා.", "target": "අපි ගමනාන්තයට ලගා වෙමු."},

{"input": "අපි මල් පූජා කරන්නෙමි.", "target": "අපි මල් පූජා කරන්නෙමු."},
{"input": "අපි මල් පූජා කලේ ය.", "target": "අපි මල් පූජා කලෙමු."},
{"input": "අපි මල් පූජා කලෝ ය.", "target": "අපි මල් පූජා කලෙමු."},
{"input": "අපි මල් පූජා කලා.", "target": "අපි මල් පූජා කලෙමු."},
{"input": "අපි මල් පූජා කරනවා.", "target": "අපි මල් පූජා කරමු."},

{"input": "අපි ගමන ආරම්භ කරන්නෙමි.", "target": "අපි ගමන ආරම්භ කරන්නෙමු."},
{"input": "අපි ගමන ආරම්භ කලේ ය.", "target": "අපි ගමන ආරම්භ කලෙමු."},
{"input": "අපි ගමන ආරම්භ කලෝ ය.", "target": "අපි ගමන ආරම්භ කලෙමු."},
{"input": "අපි ගමන ආරම්භ කලා.", "target": "අපි ගමන ආරම්භ කලෙමු."},
{"input": "අපි ගමන ආරම්භ කරනවා.", "target": "අපි ගමන ආරම්භ කරමු."},

  {"input": "අපි කුටිය අලුත් කරන්නෙමි.", "target": "අපි කුටිය අලුත් කරන්නෙමු."},
  {"input": "අපි කුටිය අලුත් කලේ ය.", "target": "අපි කුටිය අලුත් කලෙමු."},
  {"input": "අපි කුටිය අලුත් කලෝ ය.", "target": "අපි කුටිය අලුත් කලෙමු."},
  {"input": "අපි කුටිය අලුත් කලා.", "target": "අපි කුටිය අලුත් කලෙමු."},
  {"input": "අපි කුටිය අලුත් කරනවා.", "target": "අපි කුටිය අලුත් කරමු."},

  {"input": "අපි දුරකතනය පාවිච්චි කරන්නෙමි.", "target": "අපි දුරකතනය පාවිච්චි කරන්නෙමු."},
  {"input": "අපි දුරකතනය පාවිච්චි කරන්නේ ය.", "target": "අපි දුරකතනය පාවිච්චි කලෙමු."},
  {"input": "අපි දුරකතනය පාවිච්චි කරන්නෝ ය.", "target": "අපි දුරකතනය පාවිච්චි කලෙමු."},
  {"input": "අපි දුරකතනය පාවිච්චි කෙරුවා.", "target": "අපි දුරකතනය පාවිච්චි කලෙමු."},
  {"input": "අපි දුරකතනය පාවිච්චි කරනවා.", "target": "අපි දුරකතනය පාවිච්චි කරමු."},

  {"input": "අපි සෙල්ලම් කරන්න යන්නෙමි.", "target": "අපි සෙල්ලම් කරන්න යන්නෙමු."},
  {"input": "අපි සෙල්ලම් කරන්න යන්නේ ය.", "target": "අපි සෙල්ලම් කරන්න ගියෙමු."},
  {"input": "අපි සෙල්ලම් කරන්න යන්නෝ ය.", "target": "අපි සෙල්ලම් කරන්න ගියෙමු."},
  {"input": "අපි සෙල්ලම් කරන්න ගියා.", "target": "අපි සෙල්ලම් කරන්න ගියෙමු."},
  {"input": "අපි සෙල්ලම් කරන්න යනවා.", "target": "අපි සෙල්ලම් කරන්න යමු."},

  {"input": "අපි පොත් පාඩම් කරන්නෙමි.", "target": "අපි පොත් පාඩම් කරන්නෙමු."},
  {"input": "අපි පොත් පාඩම් කරන්නේ ය.", "target": "අපි පොත් පාඩම් කලෙමු."},
  {"input": "අපි පොත් පාඩම් කරන්නෝ ය.", "target": "අපි පොත් පාඩම් කලෙමු."},
  {"input": "අපි පොත් පාඩම් කලා.", "target": "අපි පොත් පාඩම් කලෙමු."},
  {"input": "අපි පොත් පාඩම් කරනවා.", "target": "අපි පොත් පාඩම් කරමු."},

  {"input": "අපි මිදුල අතු ගාන්නෙමි.", "target": "අපි මිදුල අතු ගාන්නෙමු."},
  {"input": "අපි මිදුල අතු ගාන්නේ ය.", "target": "අපි මිදුල අතු ගෑවෙමු."},
  {"input": "අපි මිදුල අතු ගාන්නෝ ය.", "target": "අපි මිදුල අතු ගෑවෙමු."},
  {"input": "අපි මිදුල අතු ගෑවා.", "target": "අපි මිදුල අතු ගෑවෙමු."},
  {"input": "අපි මිදුල අතු ගානවා.", "target": "අපි මිදුල අතු ගාමු."},

  {"input": "අපි අලුත් අවුරුද්ද සමරන්නෙමි.", "target": "අපි අලුත් අවුරුද්ද සමරන්නෙමු."},
  {"input": "අපි අලුත් අවුරුද්ද සමරන්නේ ය.", "target": "අපි අලුත් අවුරුද්ද සැමරුවෙමු."},
  {"input": "අපි අලුත් අවුරුද්ද සමරන්නෝ ය.", "target": "අපි අලුත් අවුරුද්ද සැමරුවෙමු."},
  {"input": "අපි අලුත් අවුරුද්ද සැමරුවා.", "target": "අපි අලුත් අවුරුද්ද සැමරුවෙමු."},
  {"input": "අපි අලුත් අවුරුද්ද සමරනවා.", "target": "අපි අලුත් අවුරුද්ද සමරමු."},

  {"input": "අපි පරිසරය පිරිසිදු කරන්නෙමි.", "target": "අපි පරිසරය පිරිසිදු කරන්නෙමු."},
  {"input": "අපි පරිසරය පිරිසිදු කලේ ය.", "target": "අපි පරිසරය පිරිසිදු කලෙමු."},
  {"input": "අපි පරිසරය පිරිසිදු කලෝ ය.", "target": "අපි පරිසරය පිරිසිදු කලෙමු."},
  {"input": "අපි පරිසරය පිරිසිදු කලා.", "target": "අපි පරිසරය පිරිසිදු කලෙමු."},
  {"input": "අපි පරිසරය පිරිසිදු කරනවා.", "target": "අපි පරිසරය පිරිසිදු කලෙමු."},

  {"input": "අපි අම්මව බලන්න යන්නෙමි.", "target": "අපි අම්මව බලන්න යන්නෙමු."},
  {"input": "අපි අම්මව බලන්න යන්නේ ය.", "target": "අපි අම්මව බලන්න ගියෙමු."},
  {"input": "අපි අම්මව බලන්න යන්නෝ ය.", "target": "අපි අම්මව බලන්න ගියෙමු."},
  {"input": "අපි අම්මව බලන්න ගියා.", "target": "අපි අම්මව බලන්න ගියෙමු."},
  {"input": "අපි අම්මව බලන්න යනවා.", "target": "අපි අම්මව බලන්න ගියෙමු."},

  {"input": "අපි සතුරන්ගෙන් පලිගන්න යන්නෙමි.", "target": "අපි සතුරන්ගෙන් පලිගන්න යන්නෙමු."},
  {"input": "අපි සතුරන්ගෙන් පලිගන්න යන්නේ ය.", "target": "අපි සතුරන්ගෙන් පලිගන්න ගියෙමු."},
  {"input": "අපි සතුරන්ගෙන් පලිගන්න යන්නෝ ය.", "target": "අපි සතුරන්ගෙන් පලිගන්න ගියෙමු."},
  {"input": "අපි සතුරන්ගෙන් පලිගන්න ගියා.", "target": "අපි සතුරන්ගෙන් පලිගන්න ගියෙමු."},
  {"input": "අපි සතුරන්ගෙන් පලිගන්න යනවා.", "target": "අපි සතුරන්ගෙන් පලිගන්න යමු."},

  {"input": "අපි කිසිවෙකුට කරදර නොකරන්නෙමි.", "target": "අපි කිසිවෙකුට කරදර නොකරන්නෙමු."},
  {"input": "අපි කිසිවෙකුට කරදර නොකරන්නේ ය.", "target": "අපි කිසිවෙකුට කරදර නොකලෙමු."},
  {"input": "අපි කිසිවෙකුට කරදර නොකරන්නෝ ය.", "target": "අපි කිසිවෙකුට කරදර නොකලෙමු."},
  {"input": "අපි කිසිවෙකුට කරදර නොකලා.", "target": "අපි කිසිවෙකුට කරදර නොකලෙමු."},
  {"input": "අපි කිසිවෙකුට කරදර නොකරනවා.", "target": "අපි කිසිවෙකුට කරදර නොකරමු."},

  {"input": "අපි කැලේ සංචාරය කරන්නෙමි.", "target": "අපි කැලේ සංචාරය කරන්නෙමු."},
  {"input": "අපි කැලේ සංචාරය කරන්නේ ය.", "target": "අපි කැලේ සංචාරය කලෙමු."},
  {"input": "අපි කැලේ සංචාරය කරන්නෝ ය.", "target": "අපි කැලේ සංචාරය කලෙමු."},
  {"input": "අපි කැලේ සංචාරය කලා.", "target": "අපි කැලේ සංචාරය කලෙමු."},
  {"input": "අපි කැලේ සංචාරය කරනවා.", "target": "අපි කැලේ සංචාරය කලෙමු."},

  {"input": "අපි දරුවව හොදින් බලා ගන්නෙමි.", "target": "අපි දරුවව හොදින් බලා ගන්නෙමු."},
  {"input": "අපි දරුවව හොදින් බලා ගන්නේ ය.", "target": "අපි දරුවව හොදින් බලා ගත්තෙමු."},
  {"input": "අපි දරුවව හොදින් බලා ගන්නෝ ය.", "target": "අපි දරුවව හොදින් බලා ගත්තෙමු."},
  {"input": "අපි දරුවව හොදින් බලා ගත්තා.", "target": "අපි දරුවව හොදින් බලා ගත්තෙමු."},
  {"input": "අපි දරුවව හොදින් බලා ගන්නවා.", "target": "අපි දරුවව හොදින් බලා ගමු."}

]

# Separate inputs and targets
input_texts = [pair['input'] for pair in dataset]
target_texts = [pair['target'] for pair in dataset]

# Tokenization with OOV tokens
tokenizer_input = Tokenizer(oov_token="<OOV>")
tokenizer_input.fit_on_texts(input_texts)
input_sequences = tokenizer_input.texts_to_sequences(input_texts)

tokenizer_target = Tokenizer(oov_token="<OOV>")
tokenizer_target.fit_on_texts(target_texts)
target_sequences = tokenizer_target.texts_to_sequences(target_texts)

# Pad sequences to ensure uniform length
max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_target_length, padding='post')

# Prepare data for the model
X = np.array(input_sequences)
y = np.array(target_sequences)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_input.word_index) + 1, output_dim=256, input_length=max_input_length))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(tokenizer_target.word_index) + 1, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, np.expand_dims(y_train, -1), epochs=30, batch_size=32, validation_data=(X_test, np.expand_dims(y_test, -1)))


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 197ms/step - accuracy: 0.6240 - loss: 4.5870 - val_accuracy: 0.8168 - val_loss: 1.2410
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 139ms/step - accuracy: 0.8250 - loss: 1.1771 - val_accuracy: 0.8168 - val_loss: 1.2379
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.8290 - loss: 1.0984 - val_accuracy: 0.8168 - val_loss: 1.1640
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.8281 - loss: 1.0471 - val_accuracy: 0.8168 - val_loss: 1.1173
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 222ms/step - accuracy: 0.8262 - loss: 1.0178 - val_accuracy: 0.8168 - val_loss: 1.0949
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 136ms/step - accuracy: 0.8242 - loss: 1.0077 - val_accuracy: 0.8168 - val_loss: 1.0891
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 138ms/step - accuracy: 0.8246 - loss: 0.9740 - val_accuracy: 0.8173 - val_loss: 1.0403
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 0.8335 - loss: 0.9264 - val_accuracy: 0.8370 - val_

In [2]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, np.expand_dims(y_test, -1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.9160 - loss: 0.6197
Test Accuracy: 91.35%


In [3]:
# Save the trained model to a file
model.save('model.h5')

In [4]:
import re
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from difflib import SequenceMatcher

# Load the trained model from the given path
model = load_model('/content/model.h5')

# Define the correct verbs structure
correct_verbs = {
    "මම": {
        "future": {
            "යාවි": "යන්නෙමි",
            "කාවි": "කන්නෙමි",
            "බෝවි": "බොන්නෙමි",
            "නාවි": "නාන්නෙමි",
            "නගීවි": "නගින්නෙමි",
            "බහීවි": "බසින්නෙමි",
            "අදීවි": "අදින්නෙමි",
            "කරාවි": "කරන්නෙමි",
            "ඒවි": "එන්නෙමි",
            "සෝදාවි": "සෝදන්නෙමි",
            "පිසීවි": "පිසින්නෙමි",
            "ගාවි": "ගන්නෙමි",
            "හිතාවී": "හිතන්නෙමි",
            "වේවි": "වෙන්නෙමි"
        },
        "present": {
            "යනවා": "යමි",
            "කනවා": "කමි",
            "බොනවා": "බොමි",
            "නානවා": "නාමි",
            "නගිනවා": "නගිමි",
            "බහිනවා": "බහිමි",
            "අදිනවා": "අදිමි",
            "කරනවා": "කරමි",
            "එනවා": "එමි",
            "සෝදනවා": "සෝදමි",
            "පිසිනවා": "පිසිමි",
            "ගන්නවා": "ගමි",
            "හිතනවා": "හිතමි",
            "වෙනවා": "වෙමි"
        },
        "past": {
            "යන්නේය": "ගියෙමි",
            "කෑවේය": "කෑවෙමි",
            "බීවේය": "බීවෙමි",
            "නෑවේය": "නෑවෙමි",
            "නැග්ගේය": "නැග්ගෙමි",
            "බැස්සේය": "බැස්සෙමි",
            "ඇන්දේය": "ඇන්දෙමි",
            "කරේය": "කලෙමි",
            "එන්නේය": "ආවෙමි",
            "සේදුවේය": "සේදුවෙමි",
            "පිස්සේය": "පිස්සෙමි",
            "ගන්නේය": "ගත්තෙමි",
            "හිතන්නේය": "හිතුවෙමි",
            "වුනා": "වුනෙමි"
        }
    }
}

# Function to correct words based on subject-specific verb corrections
def correct_word(word, subject=None):
    suggestions = []

    # Apply verb corrections based on the subject
    if subject and subject in correct_verbs:
        for tense, verbs in correct_verbs[subject].items():
            if word in verbs:
                corrected_word = verbs[word]
                suggestions.append(f"Replace '{word}' with '{corrected_word}'")
                word = corrected_word

    # Rule for 'ෂ' and 'ශ' patterns (additional corrections)
    if re.search(r'(ෂ|ෂෙ|ෂො|ෂේ|ෂෝ|ෂ්)න', word):
        corrected_word = re.sub(r'(ෂ|ෂෙ|ෂො|ෂේ|ෂෝ|ෂ්)න', r'\1ණ', word)
        suggestions.append(f"Replace '{word}' with '{corrected_word}'")
        word = corrected_word

    if re.search(r'(ශ|ශෙ|ශේ|ශො|ශෝ)න', word):
        corrected_word = re.sub(r'(ශ|ශෙ|ශේ|ශො|ශෝ)න', r'\1න', word)
        suggestions.append(f"Replace '{word}' with '{corrected_word}'")
        word = corrected_word

    return suggestions, word

# Function to apply rule-based corrections for a sentence
def correct_sentence(sentence):
    suggestions = []
    words = sentence.split()
    corrected_sentence = []

    # Extract the subject (first word, e.g., "මම")
    subject = words[0] if words[0] in correct_verbs else None

    for word in words:
        word_suggestions, corrected_word = correct_word(word, subject)
        if word_suggestions:
            suggestions.extend(word_suggestions)
        corrected_sentence.append(corrected_word)

    corrected_sentence = ' '.join(corrected_sentence)
    return corrected_sentence, suggestions

# Example LSTM Model Prediction
def lstm_predict(text):
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts([text])
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=50)

    predictions = model.predict(padded_sequences)
    return text  # Replace this with actual model predictions

# Main function to integrate rule-based correction with LSTM model
def correct_paragraph(paragraph):
    suggestions = []
    corrected_paragraph = []

    # Correct each sentence using rule-based system
    sentences = paragraph.split('.')
    for sentence in sentences:
        if sentence.strip():
            corrected_sentence, sentence_suggestions = correct_sentence(sentence.strip())
            suggestions.extend(sentence_suggestions)
            corrected_paragraph.append(corrected_sentence)

    corrected_paragraph = '. '.join(corrected_paragraph) + '.'

    # Pass the corrected paragraph through LSTM model
    final_paragraph = lstm_predict(corrected_paragraph)

    return final_paragraph, suggestions

# Function to calculate accuracy based on similarity
def calculate_accuracy(corrected_text, true_text):
    similarity = SequenceMatcher(None, corrected_text, true_text).ratio()
    return similarity * 100

# User interaction loop
while True:
    user_input = input("\nEnter a paragraph in Sinhala for correction (or type 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Exiting the program. Goodbye!")
        break

    true_paragraph = input("Enter the true paragraph for comparison: ")

    corrected_paragraph, suggestions = correct_paragraph(user_input)

    print(f"Corrected paragraph: {corrected_paragraph}")

    accuracy = calculate_accuracy(corrected_paragraph, true_paragraph)
    print(f"Accuracy: {accuracy:.2f}%")

    if suggestions:
        print("Suggestions:")
        for suggestion in suggestions:
            print(f"- {suggestion}")
    else:
        print("No suggestions.")



Enter a paragraph in Sinhala for correction (or type 'exit' to quit): මම සොයා ගන්නවා සහ අලුත්ම අදහසක් හිතනවා. අපි ඒවා බලාපොරොත්තු වෙනවා.
Enter the true paragraph for comparison: මම සොයා ගමි සහ අලුත්ම අදහසක් හිතමි. අපි ඒවා බලාපොරොත්තු වෙමු.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
Corrected paragraph: මම සොයා ගමි සහ අලුත්ම අදහසක් හිතමි. අපි ඒවා බලාපොරොත්තු වෙනවා.
Accuracy: 95.93%
Suggestions:
- Replace 'ගන්නවා' with 'ගමි'
- Replace 'හිතනවා' with 'හිතමි'

Enter a paragraph in Sinhala for correction (or type 'exit' to quit): මම පරීක්ෂණයට ගිහින් නදි එක්ක පන්සල් යනවා. අපි රීට පස්සෙ ගවේශණය කරනවා මොකක්ද අවුල කියලා. මම නවීන තාක්ෂණය පරික්ෂා කරනවා. එතනින් ගවේශණය පටන් ගන්නවා.
Enter the true paragraph for comparison:  මම පරීක්ෂණයට ගිහින් නදි එක්ක පන්සල් යමි. අපි රීට පස්සෙ ගවේශනය කරමු මොකක්ද අවුල කියලා. මම නවීන තාක්ෂණය පරික්ෂා කරමි. එතනින් ගවේශනය පටන් ගන්නවා.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Corrected paragraph: මම පරීක්ෂණයට ගිහින් නදි එක්ක පන්සල් යමි. අපි රීට පස්සෙ ගවේශණය කරනවා මොකක්ද අවුල කිය